In [4]:
conda install -c anaconda scikit-learn

Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

## Package Plan ##


  environment location: C:\Users\User\anaconda3

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.10.14 |                0         159 KB  anaconda
    conda-4.10.1               |   py38haa95532_1         2.9 MB
    openssl-1.1.1h             |       he774522_0         5.8 MB  anaconda
    scikit-learn-0.23.2        |   py38h47e9c7a_0         6.3 MB  anaconda
    ------------------------------------------------------------
                                           Total:        15.1 MB

The following packages will be UPDATED:

  conda              conda-forge::conda-4.10.1-py38haa244f~ --> pkgs/main::conda-4.10.1-py38haa95532_1

The following packages will be SUPERSEDED by a higher-priority chan

In [5]:
import numpy as np, pandas as pd
import ast 
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [3]:
conda install -c conda-forge spacy

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | 
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0
  - defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35ccdone

## Package Plan ##

  environment location: /Users/eshwitha/opt/anaconda3

  added / updated specs:
    - spacy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    conda-4.10.1               |   py37hf985489_0         3.0 MB  conda-forg

In [6]:
data = pd.read_csv("train_detect_sent.csv").reset_index(drop=True)

In [7]:
data.shape

(100, 14)

In [8]:
data.head(3)

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc,root_match_idx,root_match_idx_first
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"['Architecturally, the school has a Catholic c...",5,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.11010081 0.11422941 0.11560897 ... 0.0...,"[0.42473626136779785, 0.364050030708313, 0.347...","[14.563858, 15.262213, 17.398178, 14.272491, 1...",5,5,[5],5
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"['Architecturally, the school has a Catholic c...",2,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.10951651 0.11030623 0.05210006 ... -0.0...,"[0.4540746212005615, 0.32262009382247925, 0.35...","[12.889506, 12.285219, 16.843704, 8.361172, 11...",3,3,"[1, 2, 3, 4, 5, 6]",1
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"['Architecturally, the school has a Catholic c...",3,"[array([ 0.05519996, 0.05013141, 0.04787038,...",[[ 0.01195648 0.14930709 0.0266005 ... 0.0...,"[0.3958578109741211, 0.2917083501815796, 0.309...","[11.857297, 11.392319, 15.061656, 7.1847134, 8...",3,3,"[1, 2, 3, 4, 5, 6]",1


In [9]:
ast.literal_eval(data["sentences"][0])

['Architecturally, the school has a Catholic character.',
 "Atop the Main Building's gold dome is a golden statue of the Virgin Mary.",
 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".',
 'Next to the Main Building is the Basilica of the Sacred Heart.',
 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.',
 'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.',
 'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.']

In [10]:
data = data[data["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

In [11]:
# for each sentence, listing the euc dt and cosine sim from target sentence
def create_features(data):
    train = pd.DataFrame()
     
    for k in range(len(data["euclidean_dis"])):
        dis = ast.literal_eval(data["euclidean_dis"][k])
        for i in range(len(dis)):
            train.loc[k, "column_euc_"+"%s"%i] = dis[i]
    
    print("Finished")
    
    for k in range(len(data["cosine_sim"])):
        dis = ast.literal_eval(data["cosine_sim"][k].replace("nan","1"))
        for i in range(len(dis)):
            train.loc[k, "column_cos_"+"%s"%i] = dis[i]
            
    train["target"] = data["target"]
    return train

In [12]:
train = create_features(data)

Finished


In [13]:
del data

In [14]:
train.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,14.563858,15.262213,17.398178,14.272491,13.339654,9.336262,15.720997,NaN,NaN,NaN,...,0.364050,0.347755,0.394242,0.371025,0.185690,0.351921,NaN,NaN,NaN,5
1,12.889506,12.285219,16.843704,8.361172,11.918098,17.601220,14.929258,NaN,NaN,NaN,...,0.322620,0.355004,0.271561,0.392342,0.384383,0.362597,NaN,NaN,NaN,2
2,11.857297,11.392319,15.061656,7.184713,8.465475,13.927309,12.249869,NaN,NaN,NaN,...,0.291708,0.309919,0.223061,0.265975,0.293025,0.288712,NaN,NaN,NaN,3


In [15]:
train.head(3).transpose()

,0,1,2
column_euc_0,14.563858,12.889506,11.857297
column_euc_1,15.262213,12.285219,11.392319
column_euc_2,17.398178,16.843704,15.061656
column_euc_3,14.272491,8.361172,7.184713
column_euc_4,13.339654,11.918098,8.465475
column_euc_5,9.336262,17.601220,13.927309
column_euc_6,15.720997,14.929258,12.249869
column_euc_7,NaN,NaN,NaN
column_euc_8,NaN,NaN,NaN
column_euc_9,NaN,NaN,NaN


### Fitting Multinomial Logistic Regression

### Standardize

In [16]:
train.apply(max, axis = 0)

column_euc_0    22.576950
column_euc_1    30.682000
column_euc_2    22.579485
column_euc_3    25.461891
column_euc_4    18.283308
column_euc_5    18.323954
column_euc_6    25.030850
column_euc_7          NaN
column_euc_8          NaN
column_euc_9          NaN
column_cos_0     0.587923
column_cos_1     1.008395
column_cos_2     0.756976
column_cos_3     0.471499
column_cos_4     0.590982
column_cos_5     0.551157
column_cos_6     0.472575
column_cos_7          NaN
column_cos_8          NaN
column_cos_9          NaN
target           9.000000
dtype: float64

In [17]:
subset1 = train.iloc[:,:10].fillna(60)
subset2 = train.iloc[:,10:].fillna(1)

In [18]:
 subset1.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9
0,14.563858,15.262213,17.398178,14.272491,13.339654,9.336262,15.720997,60.0,60.0,60.0
1,12.889506,12.285219,16.843704,8.361172,11.918098,17.601220,14.929258,60.0,60.0,60.0
2,11.857297,11.392319,15.061656,7.184713,8.465475,13.927309,12.249869,60.0,60.0,60.0


In [19]:
 subset2.head(3)

,column_cos_0,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,0.424736,0.364050,0.347755,0.394242,0.371025,0.185690,0.351921,1.0,1.0,1.0,5
1,0.454075,0.322620,0.355004,0.271561,0.392342,0.384383,0.362597,1.0,1.0,1.0,2
2,0.395858,0.291708,0.309919,0.223061,0.265975,0.293025,0.288712,1.0,1.0,1.0,3


In [20]:
train2 = pd.concat([subset1, subset2],axis=1)
train2 = train2.reindex(subset1.index)

In [21]:
train2.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,14.563858,15.262213,17.398178,14.272491,13.339654,9.336262,15.720997,60.0,60.0,60.0,...,0.364050,0.347755,0.394242,0.371025,0.185690,0.351921,1.0,1.0,1.0,5
1,12.889506,12.285219,16.843704,8.361172,11.918098,17.601220,14.929258,60.0,60.0,60.0,...,0.322620,0.355004,0.271561,0.392342,0.384383,0.362597,1.0,1.0,1.0,2
2,11.857297,11.392319,15.061656,7.184713,8.465475,13.927309,12.249869,60.0,60.0,60.0,...,0.291708,0.309919,0.223061,0.265975,0.293025,0.288712,1.0,1.0,1.0,3


In [22]:
train2.apply(max, axis = 0)

column_euc_0    22.576950
column_euc_1    30.682000
column_euc_2    22.579485
column_euc_3    25.461891
column_euc_4    60.000000
column_euc_5    60.000000
column_euc_6    60.000000
column_euc_7    60.000000
column_euc_8    60.000000
column_euc_9    60.000000
column_cos_0     0.587923
column_cos_1     1.008395
column_cos_2     0.756976
column_cos_3     0.471499
column_cos_4     1.000000
column_cos_5     1.000000
column_cos_6     1.000000
column_cos_7     1.000000
column_cos_8     1.000000
column_cos_9     1.000000
target           9.000000
dtype: float64

In [23]:
scaler = MinMaxScaler()
X = scaler.fit_transform(train2.iloc[:,:-1])

In [24]:
X

array([[0.51319541, 0.33984604, 0.64407879, ..., 1.        , 1.        ,
        1.        ],
       [0.4114766 , 0.21239458, 0.60599013, ..., 1.        , 1.        ,
        1.        ],
       [0.34876871, 0.17416763, 0.48357532, ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.38197757, 0.07869259, 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.16273841, 0.107725  , 0.2148866 , ..., 1.        , 1.        ,
        1.        ],
       [0.30719658, 0.09616829, 0.1410048 , ..., 1.        , 1.        ,
        1.        ]])

In [25]:
train_x, test_x, train_y, test_y = train_test_split(X, train.iloc[:,-1], train_size=0.8, random_state = 5)

In [26]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
mul_lr.fit(train_x, train_y)

print("Multinomial Logistic regression Train Accuracy : ", metrics.accuracy_score(train_y, mul_lr.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", metrics.accuracy_score(test_y, mul_lr.predict(test_x)))


Multinomial Logistic regression Train Accuracy :  0.5
Multinomial Logistic regression Test Accuracy :  0.2777777777777778


### Random Forest

In [27]:
rf = RandomForestClassifier(min_samples_leaf=8, n_estimators=60)
rf.fit(train_x, train_y)

print("Random Forest Train Accuracy : ", metrics.accuracy_score(train_y, rf.predict(train_x)))
print("Random Forest Test Accuracy : ", metrics.accuracy_score(test_y, rf.predict(test_x)))

Random Forest Train Accuracy :  0.625
Random Forest Test Accuracy :  0.3333333333333333
